In [1]:
import numpy as np
from scipy import sparse

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.utils.validation import check_is_fitted, _check_feature_names_in
from sklearn.preprocessing import OneHotEncoder
import sklearn
import sklearn.impute

import pandas as pd
from pandas.api.types import is_numeric_dtype
import sklearn.compose

import torch



In [2]:
dataset_file = '/Users/gabrielketron/tpot2_addimputers/tpot2/ImputerExperiments/data/Spam.csv'
#%% System Parameters
# 1. Mini batch size
mb_size = 128
# 2. Missing rate
p_miss = 0.2
# 3. Hint rate
p_hint = 0.9
# 4. Loss Hyperparameters
alpha = 10
# 5. Train Rate
train_rate = 0.8

#%% Data

# Data generation
Data = np.loadtxt(dataset_file, delimiter=",",skiprows=1)

# Parameters
No = len(Data)
Dim = len(Data[0,:])

# Hidden state dimensions
H_Dim1 = Dim
H_Dim2 = Dim

# Normalization (0 to 1)
Min_Val = np.zeros(Dim)
Max_Val = np.zeros(Dim)

for i in range(Dim):
    Min_Val[i] = np.min(Data[:,i])
    Data[:,i] = Data[:,i] - np.min(Data[:,i])
    Max_Val[i] = np.max(Data[:,i])
    Data[:,i] = Data[:,i] / (np.max(Data[:,i]) + 1e-6)    

#%% Missing introducing
p_miss_vec = p_miss * np.ones((Dim,1)) 
   
Missing = np.zeros((No,Dim))

for i in range(Dim):
    A = np.random.uniform(0., 1., size = [len(Data),])
    B = A > p_miss_vec[i]
    Missing[:,i] = 1.*B

    
#%% Train Test Division    
   
idx = np.random.permutation(No)

Train_No = int(No * train_rate)
Test_No = No - Train_No
    
# Train / Test Features
trainX = Data[idx[:Train_No],:]
testX = Data[idx[Train_No:],:]

# Train / Test Missing Indicators
trainM = Missing[idx[:Train_No],:]
testM = Missing[idx[Train_No:],:]

In [3]:
class GainImputer(BaseEstimator, TransformerMixin):
    """
    Base class for all imputers.
    It adds automatically support for `add_indicator`.
    """

    def __init__(self, 
                 batch_size=128, 
                 hint_rate=0.9, 
                 alpha=100, 
                 iterations=10000,
                 missing_values=np.nan, 
                 random_state=None):
        self.batch_size = batch_size
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.iterations = iterations
        self.missing_values = missing_values
        self.random_state = random_state
        self.device = (
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
            )
        if random_state is not None:
            torch.manual_seed(self.random_state)

        
        

    def fit(self, X, y=None):
        self.fit_transform(X)
        return self

    def transform(self, X):
        if hasattr(X, 'dtypes'):
            X = X.to_numpy()
        #define mask matrix
        X_mask = 1 - np.isnan(X)
        #get dimensions
        no, dim = X.shape
        int_dim = int(dim)
        #normalize the original data, and save parameters for renormalization
        norm_data = X.copy()
        min_val = np.zeros(dim)
        max_val = np.zeros(dim)
        for i in range(dim):
            min_val[i] = np.nanmin(norm_data[i])
            norm_data[:, i] -= np.nanmin(norm_data[:, i])
            max_val[i] = np.nanmax(norm_data[i])
            norm_data[:, i] /= (np.nanmax(norm_data[:, i]) + 1e-06)
        norm_parameters = {'min_val': min_val, 'max_val': max_val}
        norm_data_filled = np.nan_to_num(norm_data, 0)

        Z_mb = self._uniform_sampler(0, 0.01, no, dim)
        M_mb = X_mask
        X_mb = norm_data_filled
        New_X_mb = M_mb * X_mb + (1 - M_mb)*Z_mb

        X_mb = torch.tensor(X_mb, dtype=torch.float32)
        M_mb = torch.tensor(M_mb, dtype=torch.float32)
        New_X_mb = torch.tensor(New_X_mb, dtype=torch.float32)
        #test loss
        G_sample = self.modelG.G_prob(New_X_mb, M_mb)
        MSE_final = torch.mean(((1-M_mb)*X_mb-(1-M_mb)*G_sample)**2)/ torch.mean(1-M_mb)
        print('Final Test RMSE: ' + str(np.sqrt(MSE_final.item())))
        imputed_data = M_mb * X_mb + (1-M_mb) * G_sample
        imputed_data = imputed_data.cpu().detach().numpy()
        _, dim = imputed_data.shape
        renorm_data = imputed_data.copy()
        for i in range(dim):
            renorm_data[:,i] = renorm_data[:,i] * (max_val[i] + 1e-6)   
            renorm_data[:,i] = renorm_data[:,i] + min_val[i]
        for i in range(dim):
            temp = X[~np.isnan(X[:, i]), i]
            # Only for the categorical variable
            if len(np.unique(temp)) < 20:
                renorm_data[:, i] = np.round(renorm_data[:, i])
        return renorm_data
        
    def fit_transform(self, X, y=None):
        if hasattr(X, 'dtypes'):
            X = X.to_numpy()
        #define mask matrix
        X_mask = 1 - np.isnan(X)
        #get dimensions
        no, dim = X.shape
        int_dim = int(dim)
        #normalize the original data, and save parameters for renormalization
        norm_data = X.copy()
        min_val = np.zeros(dim)
        max_val = np.zeros(dim)
        for i in range(dim):
            min_val[i] = np.nanmin(norm_data[i])
            norm_data[:, i] -= np.nanmin(norm_data[:, i])
            max_val[i] = np.nanmax(norm_data[i])
            norm_data[:, i] /= (np.nanmax(norm_data[:, i]) + 1e-06)
        norm_parameters = {'min_val': min_val, 'max_val': max_val}
        norm_data_filled = np.nan_to_num(norm_data, 0)
        #Torch version of Gain
        # Initalize discriminator weights, gives hints and data as inputs
        D_W1 = torch.tensor(self._xavier_init([dim*2, int_dim]), dtype=torch.float32) 
        D_b1 = torch.zeros(size=[int_dim], dtype=torch.float32)
        D_W2 = torch.tensor(self._xavier_init([int_dim, int_dim]), dtype=torch.float32)
        D_b2 = torch.zeros(size=[int_dim], dtype=torch.float32)
        D_W3 = torch.tensor(self._xavier_init([int_dim, dim]), dtype=torch.float32)
        D_b3 = torch.zeros(size=[dim], dtype=torch.float32)
        theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]
        self.modelD = self.Discriminator(theta_D).to(self.device)
        # Initalize generator weights, gives hints and data as inputs
        G_W1 = torch.tensor(self._xavier_init([dim*2, int_dim]), dtype=torch.float32) 
        G_b1 = torch.zeros(size=[int_dim], dtype=torch.float32)
        G_W2 = torch.tensor(self._xavier_init([int_dim, int_dim]), dtype=torch.float32)
        G_b2 = torch.zeros(size=[int_dim], dtype=torch.float32)
        G_W3 = torch.tensor(self._xavier_init([int_dim, dim]), dtype=torch.float32)
        G_b3 = torch.zeros(size=[dim], dtype=torch.float32)
        theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]
        self.modelG = self.Generator(theta_G).to(self.device)
        #Data + Mask as inputs (Random noise is in missing components)
        def discriminator_loss(M, New_X, H):
            G_sample = self.modelG.G_prob(New_X, M)
            Hat_New_X = New_X * M + G_sample * (1-M)
            D_prob = self.modelD.D_prob(Hat_New_X, H)
            D_loss = -torch.mean(M*torch.log(D_prob + 1e-8) 
                                 + (1-M)*torch.log(1. - D_prob + 1e-8))
            return D_loss
        def generator_loss(X, M, New_X, H):
            G_sample = self.modelG.G_prob(New_X, M)
            Hat_New_X = New_X * M + G_sample * (1-M)
            D_prob = self.modelD.D_prob(Hat_New_X, H)
            G_loss = -torch.mean((1-M)*torch.log(D_prob + 1e-8))
            MSE_train_loss = torch.mean(
                (M*New_X - M*G_sample)**2
                ) / torch.mean(M)
            G_loss_final = G_loss + self.alpha*MSE_train_loss
            MSE_test_loss = torch.mean(
                ((1-M)*X-(1-M)*G_sample)**2
                )/ torch.mean(1-M)
            return G_loss_final, MSE_train_loss, MSE_test_loss
        optimizer_D = torch.optim.Adam(params=self.modelD.parameters())
        optimizer_G = torch.optim.Adam(params=self.modelG.parameters())
        #Training Iterations
        for it in range(self.iterations):
            batch_idx = self._sample_batch_index(no, self.batch_size)
            X_mb = norm_data_filled[batch_idx, :]
            M_mb = X_mask[batch_idx, :]
            #sample random vectors
            Z_mb = self._uniform_sampler(0, 0.01, self.batch_size, dim)
            #Sample hint vectors
            H_mb_temp = self._binary_sampler(self.hint_rate, 
                                             self.batch_size, dim)
            H_mb = M_mb * H_mb_temp
            #combine vectors with observed vectors
            New_X_mb = M_mb*X_mb + (1-M_mb)*Z_mb #Introduce Missin Data

            X_mb = torch.tensor(X_mb, dtype=torch.float32)
            M_mb = torch.tensor(M_mb, dtype=torch.float32)
            H_mb = torch.tensor(H_mb, dtype=torch.float32)
            New_X_mb = torch.tensor(New_X_mb, dtype=torch.float32)

            optimizer_D.zero_grad()
            D_loss_curr = discriminator_loss(M=M_mb, New_X=New_X_mb, H=H_mb)
            D_loss_curr.requires_grad = True
            D_loss_curr.backward()
            optimizer_D.step()
            optimizer_G.zero_grad()
            G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = generator_loss(X=X_mb, M=M_mb, New_X=New_X_mb, H=H_mb)
            G_loss_curr.requires_grad = True
            G_loss_curr.backward()
            optimizer_G.step()

            if it % 100 == 0:
                print('Iter: {}'.format(it))
                print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr.item())))
                print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr.item())))
                print()
            

        Z_mb = self._uniform_sampler(0, 0.01, no, dim)
        M_mb = X_mask
        X_mb = norm_data_filled
        New_X_mb = M_mb * X_mb + (1 - M_mb)*Z_mb

        X_mb = torch.tensor(X_mb, dtype=torch.float32)
        M_mb = torch.tensor(M_mb, dtype=torch.float32)
        New_X_mb = torch.tensor(New_X_mb, dtype=torch.float32)

        print(f'M_mb size: {M_mb.shape}')
        print(f'New_X_mb size: {New_X_mb.shape}')
        #test loss
        G_sample = self.modelG.G_prob(New_X_mb, M_mb)
        MSE_final = torch.mean(((1-M_mb)*X_mb-(1-M_mb)*G_sample)**2)/ torch.mean(1-M_mb)
        print('Final Test RMSE: ' + str(np.sqrt(MSE_final.item())))
        imputed_data = M_mb * X_mb + (1-M_mb) * G_sample
        imputed_data = imputed_data.cpu().detach().numpy()
        _, dim = imputed_data.shape
        renorm_data = imputed_data.copy()
        for i in range(dim):
            renorm_data[:,i] = renorm_data[:,i] * (max_val[i] + 1e-6)   
            renorm_data[:,i] = renorm_data[:,i] + min_val[i]
        for i in range(dim):
            temp = X[~np.isnan(X[:, i]), i]
            # Only for the categorical variable
            if len(np.unique(temp)) < 20:
                renorm_data[:, i] = np.round(renorm_data[:, i])
        return renorm_data
    
    def _binary_sampler(self, p, rows, cols):
        '''Sample binary random variables.
        Args:
            - p: probability of 1
            - rows: the number of rows
            - cols: the number of columns
        Returns:
            - binary_random_matrix: generated binary random matrix.
        '''
        unif_random_matrix = np.random.uniform(0., 1., size = [rows, cols])
        binary_random_matrix = 1*(unif_random_matrix > p)
        return binary_random_matrix

    def _uniform_sampler(self, low, high, rows, cols):
        '''Sample uniform random variables.
        Args:
            - low: low limit
            - high: high limit
            - rows: the number of rows
            - cols: the number of columns
        Returns:
            - uniform_random_matrix: generated uniform random matrix.
        '''
        return np.random.uniform(low, high, size = [rows, cols])       

    def _sample_batch_index(self, total, batch_size):
        '''Sample index of the mini-batch.
        Args:
            - total: total number of samples
            - batch_size: batch size
        Returns:
            - batch_idx: batch index
        '''
        total_idx = np.random.permutation(total)
        batch_idx = total_idx[:batch_size]
        return batch_idx
    
    def _xavier_init(self, size):
            '''Xavier initialization.
            Args:
                - size: vector size
            Returns:
                - initialized random vector.
            '''
            in_dim = size[0]
            xavier_stddev = 1./np.sqrt(in_dim / 2.)
            return np.random.normal(scale = xavier_stddev, size=size)
    
    class Generator(torch.nn.Module):
        def __init__(self, params):
            super().__init__()
            self.theta_G = params
            self.G_W1 = self.theta_G[0] 
            self.G_b1 = self.theta_G[3]
            self.G_W2 = self.theta_G[1] 
            self.G_b2 = self.theta_G[4]
            self.G_W3 = self.theta_G[2]
            self.G_b3 = self.theta_G[5]

        def G_prob(self, X: torch.float32, M: torch.float32):
            inputs = torch.concat([X, M], dim=1)
            G_h1 = torch.nn.functional.relu(
                torch.matmul(inputs, self.G_W1) + self.G_b1
                )
            G_h2 = torch.nn.functional.relu(
                torch.matmul(G_h1, self.G_W2) + self.G_b2
                )
            g_prob = torch.nn.functional.sigmoid(
                torch.matmul(G_h2, self.G_W3) + self.G_b3
                )
            return g_prob
        
    class Discriminator(torch.nn.Module):
        def __init__(self, params):
            super().__init__()
            self.theta_D = params
            self.D_W1 = self.theta_D[0] 
            self.D_b1 = self.theta_D[3]
            self.D_W2 = self.theta_D[1] 
            self.D_b2 = self.theta_D[4]
            self.D_W3 = self.theta_D[2]
            self.D_b3 = self.theta_D[5]
        
        def D_prob(self, X: torch.float32, H: torch.float32):
            inputs = torch.concat([X, H], dim=1)
            D_h1 = torch.nn.functional.relu(
                torch.matmul(inputs, self.D_W1) + self.D_b1
                )
            D_h2 = torch.nn.functional.relu(
                torch.matmul(D_h1, self.D_W2) + self.D_b2
                )
            d_prob = torch.nn.functional.sigmoid(
                torch.matmul(D_h2, self.D_W3) + self.D_b3
                )
            return d_prob


In [4]:
gain = GainImputer(batch_size=128, hint_rate=0.9, alpha=10, iterations=1000)

gain.fit(trainX)

out = gain.transform(trainX)

def rmse_loss(ori_data, imputed_data, data_m):
        '''Compute RMSE loss between ori_data and imputed_data
        Args:
            - ori_data: original data without missing values
            - imputed_data: imputed data
            - data_m: indicator matrix for missingness
        Returns:
            - rmse: Root Mean Squared Error
        '''
        #ori_data, norm_parameters = normalization(ori_data)
        #imputed_data, _ = normalization(imputed_data, norm_parameters)
        # Only for missing values
        nominator = np.sum(((1-data_m) * ori_data - (1-data_m) * imputed_data)**2)
        denominator = np.sum(1-data_m)
        rmse = np.sqrt(nominator/float(denominator))
        return rmse

print(rmse_loss(ori_data=trainX, imputed_data=out, data_m=trainM))
print(out)

ValueError: optimizer got an empty parameter list